In [19]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.multicomp
from itertools import combinations

datafilename = "data_processed-without-outliers.csv"
d = pd.read_csv(datafilename)
print("Loading", len(d), "lines of data (outliers removed).")

resultsfilename = "data_stats-without-outliers.csv"
st = pd.read_csv(resultsfilename)
print("Loading", len(st), "lines of stats.")

# ------------------------------------------------------
# functions to add eta squared and omega squared
# to the ANOVA summary table

def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov

def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov

Loading 14520 lines of data (outliers removed).
Loading 808 lines of stats.


In [28]:
factors = ["StudyID", "Training", "Font", "Firstfont"]
metrics = ["RT", "AUC", "AUC_word"]

def anova(factor, metrics, no):
    for f in combinations(factors, no):
        if f[0] != f[1]:
            f_ = ["C(%s)" % x for x in f]
            for metric in metrics:
                formula = metric + " ~ " + (" * ".join(f_))
                model = ols(formula, st).fit()
                aov_table = anova_lm(model, typ=2)
                eta_squared(aov_table)
                omega_squared(aov_table)
                print()
                print("# %s (metric: %s)" % (f, metric))
                print()
                display(aov_table)
                # overall model significance
                print(f"Overall model F(%d, %d) = %.3f, p = %.4f" % (model.df_model, model.df_resid, model.fvalue, model.f_pvalue))
                print()

anova(factors, metrics, 2)
print(30 * "_")
anova(factors, metrics, 3)

# ('StudyID', 'Training') (metric: RT)



,sum_sq,df,F,PR(>F),eta_sq,omega_sq
C(StudyID),1.983269e+04,1.0,0.021936,0.882295,0.000026,-0.001171
C(Training),2.375288e+07,4.0,6.567933,0.000033,0.031498,0.026670
C(StudyID):C(Training),8.845468e+06,4.0,2.445869,0.045115,0.011730,0.006926
Residual,7.214904e+08,798.0,NaN,NaN,NaN,NaN


Overall model F(9, 798) = 4.030, p = 0.0000

# ('StudyID', 'Training') (metric: AUC)



,sum_sq,df,F,PR(>F),eta_sq,omega_sq
C(StudyID),0.182620,1.0,16.082524,0.000066,0.019118,0.017908
C(Training),0.190207,4.0,4.187671,0.002313,0.019912,0.015139
C(StudyID):C(Training),0.118228,4.0,2.602952,0.034819,0.012377,0.007613
Residual,9.061438,798.0,NaN,NaN,NaN,NaN


Overall model F(9, 798) = 5.366, p = 0.0000

# ('StudyID', 'Training') (metric: AUC_word)



...

Overall model F(19, 788) = 3.595, p = 0.0000

# ('Training', 'Font', 'Firstfont') (metric: AUC)



,sum_sq,df,F,PR(>F),eta_sq,omega_sq
C(Training),0.249878,4.0,5.496347,0.000229,0.025992,0.021238
C(Font),0.000395,1.0,0.034732,0.852207,0.000041,-0.001140
C(Firstfont),0.015453,1.0,1.359623,0.243956,0.001607,0.000425
C(Training):C(Font),0.018744,4.0,0.412296,0.799861,0.001950,-0.002776
C(Training):C(Firstfont),0.334276,4.0,7.352771,0.000008,0.034771,0.030007
C(Font):C(Firstfont),0.018778,1.0,1.652171,0.199041,0.001953,0.000770
C(Training):C(Font):C(Firstfont),0.019941,4.0,0.438621,0.780749,0.002074,-0.002652
Residual,8.956125,788.0,NaN,NaN,NaN,NaN


Overall model F(19, 788) = 3.027, p = 0.0000

# ('Training', 'Font', 'Firstfont') (metric: AUC_word)



,sum_sq,df,F,PR(>F),eta_sq,omega_sq
C(Training),0.128196,4.0,1.552465,0.186372,0.015025,0.005334
C(Font),0.000087,1.0,0.004215,0.948267,0.000010,-0.002404
C(Firstfont),0.007135,1.0,0.345629,0.556943,0.000836,-0.001579
C(Training):C(Font),0.082216,4.0,0.995648,0.409757,0.009636,-0.000042
C(Training):C(Firstfont),0.202577,4.0,2.453225,0.045517,0.023743,0.014031
C(Font):C(Firstfont),0.058236,1.0,2.820962,0.093854,0.006826,0.004395
C(Training):C(Font):C(Firstfont),0.126330,4.0,1.529869,0.192745,0.014807,0.005116
Residual,7.927265,384.0,NaN,NaN,NaN,NaN


Overall model F(19, 384) = 1.527, p = 0.0729

